In [1]:
import librosa
import os
import numpy as np
from sklearn.model_selection import train_test_split

In [6]:
DATA_DIR = "/Speech Emotion Recognition/EmoDB/wav"

In [5]:
def extract_features(file_path, sr=22050):
    try:
        y, sr = librosa.load(file_path, sr=sr)
        y, _ = librosa.effects.trim(y)

        feats = []

        # MFCC (static)
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
        feats += [np.mean(mfcc, axis=1), np.std(mfcc, axis=1), np.min(mfcc, axis=1), np.max(mfcc, axis=1)]

        # Delta MFCC (dynamic → mean only)
        feats += [
            np.mean(librosa.feature.delta(mfcc), axis=1),
            np.mean(librosa.feature.delta(mfcc, order=2), axis=1)
        ]

        # Chroma (static)
        chroma = librosa.feature.chroma_stft(y=y, sr=sr)
        feats.append(np.mean(chroma, axis=1))

        # Mel spectrogram (static distribution)
        mel = librosa.power_to_db(librosa.feature.melspectrogram(y=y, sr=sr))
        feats += [np.mean(mel, axis=1), np.std(mel, axis=1)]

        # Tonnetz (static harmony)
        tonnetz = librosa.feature.tonnetz(y=librosa.effects.harmonic(y), sr=sr)
        feats.append(np.mean(tonnetz, axis=1))

        # Spectral features (dynamic)
        for f in [
            librosa.feature.spectral_centroid,
            librosa.feature.spectral_bandwidth,
            librosa.feature.spectral_rolloff,
            librosa.feature.spectral_flatness
        ]:
            v = f(y=y, sr=sr) if f != librosa.feature.spectral_flatness else f(y=y)
            feats += [np.mean(v, axis=1), np.std(v, axis=1)]

        # Zero Crossing Rate (dynamic)
        zcr = librosa.feature.zero_crossing_rate(y)
        feats += [np.mean(zcr, axis=1), np.std(zcr, axis=1)]

        return np.hstack([np.hstack(f) for f in feats])
    except Exception as e:
        print(f"Error processing {file_path}: {e}")


In [ ]:
emotion_map_en = {
    "A": "anger",
    "B": "boredom",
    "D": "disgust",
    "F": "anxiety/fear",
    "H": "happiness",
    "S": "sadness",
    "N": "neutral"
}

for actor in os.listdir(DATA_DIR):
    for file in os.listdir(os.path.join(DATA_DIR, actor)):
        code = 